# Gestor de investimento
---

In [1]:
import pandas as pd
import numpy as np
import os


# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df_abr = pd.read_excel('../datasets/PosicaoHistorico_200091-3.xls')
df_jan = pd.read_excel('../datasets/PosicaoHistorico_200091-4.xls')

WARNING *** file size (65692) not 512 + multiple of sector size (512)
WARNING *** file size (66716) not 512 + multiple of sector size (512)


In [16]:
# Função para pegar os dados de Papel (ações)
def get_stocks(df):
    stocks = []

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Papel' or row['Unnamed: 2'] == 'Ações':
            continue
        elif row['Unnamed: 2'] == 'Opções':
            break
        else:
            stocks.append(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18', 
                                'Unnamed: 28', 'Unnamed: 35', 'Unnamed: 43', 
                                'Unnamed: 51', 'Unnamed: 61', 'Unnamed: 68', 
                                'Unnamed: 76', 'Unnamed: 83']].values)

    pd_stock = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetado', 'Qtd Dia', 
                                    'Qtd Garantia BOV', 'Qtd Garantia BMF', 
                                    'Qtd Estruturados', 'Liq Termo', 
                                    'Qtd Total', 'Cotacao', 'Financeiro'], data=stocks)

    return pd_stock


def get_picking(df):
    picking = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Proventos de Ação':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Papel':
            continue
        elif row['Unnamed: 2'] == 'Renda Fixa':
            break
        elif start == True:
            picking.append(row[['Unnamed: 2', 'Unnamed: 11', 
                            'Unnamed: 22', 'Unnamed: 60', 
                            'Unnamed: 77']].values)

    pd_picking = pd.DataFrame(columns=['Papel', 'Qtd Provisionada', 
                                    'Tipo', 'Data Pagamento', 'Valor'], data=picking)
  
    return pd_picking


def get_fi(df):
    x = []
    start = False

    for index, row in df[~df.isnull()].iterrows():
        if row['Unnamed: 2'] == 'Fundos de Investimentos':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome Fundo':
            continue
        elif row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
            break
        elif (start == True) and ((type(row['Unnamed: 2']) == str)):
            if type(row['Unnamed: 2']) == str: 
                fi_name = np.array([row['Unnamed: 2']])
                data = df[df.index == index +1][['Unnamed: 13', 'Unnamed: 19', 
                                                    'Unnamed: 33', 'Unnamed: 40', 
                                                    'Unnamed: 48', 'Unnamed: 54', 
                                                    'Unnamed: 60', 'Unnamed: 70', 
                                                    'Unnamed: 81']]
                
                data.insert(0, 'Nome FI', row['Unnamed: 2'])
                # print(data.values[0])
                x.append(data.values[0])
                #display(data.head())
    
    pd_fi = pd.DataFrame(columns=['Nome', 'Data', 'Qtd Cotas', 
                                    'Valor Cota', 'Valor Bruto', 
                                    'IR', 'IOF', 'Valor Liquido', 
                                    'Aplicacao Pendente', 'Total Bruto'], data=x)
    return pd_fi


def get_fii(df):
    fii = []
    start = False

    for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
        if row['Unnamed: 2'] == 'Posição de Fundos Imobiliários':
                start = True
                continue
        elif row['Unnamed: 2'] == 'Nome':
            continue
        elif row['Unnamed: 2'] == 'Proventos de Fundo Imobiliário':
            break
        elif start == True:
            fii.append(row[['Unnamed: 2', 'Unnamed: 14', 
                            'Unnamed: 26', 'Unnamed: 38', 
                            'Unnamed: 45', 'Unnamed: 55', 
                            'Unnamed: 74']].values)

    df_result = pd.DataFrame(columns=['Papel', 'Qtd Disponivel', 
                                    'Qtd Projetada', 'Qtd Dia', 'Qtde Total', 
                                    'Ult Cotacao', 'Financeiro'], data=fii)
  
    return df_result

In [20]:
files = os.listdir('../datasets/')

stocks = pd.DataFrame()
pickings = pd.DataFrame()
fis = pd.DataFrame()
fiis = pd.DataFrame()

for file_name in files:
    
    if file_name == '.DS_Store':
        continue
    
    df = pd.read_excel('../datasets/' + file_name)
    date_position = df[df['Unnamed: 56'].str.contains('Data de referência', na=False)]['Unnamed: 56']
    
    # position date
    date_position = pd.to_datetime(date_position.str.replace('Data de referência: ', ''), format='%d/%m/%Y')
    month = int(date_position.dt.month.values)
    year = int(date_position.dt.year.values)
    period = str(year) + '/' + str(month)

    # get stocks
    df_stocks = get_stocks(df)
    #display(df_stocks.head())
    df_stocks['period'] = period
    stocks = stocks.append(df_stocks, ignore_index=True)

    df_pickings = get_picking(df)
    df_pickings['period'] = period
    pickings = pickings.append(df_pickings, ignore_index=True)

    df_fi = get_fi(df)
    df_fi['period'] = period
    fis = fis.append(df_fi, ignore_index=True)

    df_fii = get_fii(df)
    df_fii['period'] = period
    fiis = fiis.append(df_fii, ignore_index=True)

# display(stocks.sort_values('period', ascending=False))
# display(fi)

WARNING *** file size (71838) not 512 + multiple of sector size (512)
WARNING *** file size (60572) not 512 + multiple of sector size (512)
WARNING *** file size (69788) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (71834) not 512 + multiple of sector size (512)
WARNING *** file size (65692) not 512 + multiple of sector size (512)
WARNING *** file size (70812) not 512 + multiple of sector size (512)
WARNING *** file size (65696) not 512 + multiple of sector size (512)
WARNING *** file size (66716) not 512 + multiple of sector size (512)
WARNING *** file size (60572) not 512 + multiple of sector size (512)
WARNING *** file size (65180) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)


In [11]:
fis['period'].sort_values().unique()

array(['2019/10', '2019/11', '2019/12', '2019/4', '2019/5', '2019/6',
       '2019/7', '2019/8', '2019/9', '2020/1', '2020/2', '2020/3',
       '2020/4'], dtype=object)

In [22]:
display(stocks.head())
display(pickings.head())
display(fis.head())
display(fiis.head())

,Papel,Qtd Disponivel,Qtd Projetado,Qtd Dia,Qtd Garantia BOV,Qtd Garantia BMF,Qtd Estruturados,Liq Termo,Qtd Total,Cotacao,Financeiro,period
0,BPAC11,200,0,0,0,0,0,0,200,42.30,"8,460.00",2020/4
1,JHSF3,1600,0,0,0,0,0,0,1600,4.16,"6,656.00",2020/4
2,MGLU3,800,0,0,0,0,0,0,800,49.70,"39,760.00",2020/4
3,PETR4,400,0,0,0,0,0,0,400,18.05,"7,220.00",2020/4
4,WEGE3,300,0,0,0,0,0,0,300,39.94,"11,982.00",2020/4


,Papel,Qtd Provisionada,Tipo,Data Pagamento,Valor,period
0,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020/4
1,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020/4
2,WEGE3,300,JUROS SOBRE CAPITAL PROPRIO,12/08/2020,9.07,2020/4
3,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,28.63,2020/2
4,MGLU3,800,JUROS SOBRE CAPITAL PROPRIO,Indeterminado,58.88,2020/2


,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period
0,XP Long Short FIC FIM,2020-04-30,"6,902.02",2.84,"19,586.10",0.00,0,"19,586.10",0,"19,586.10",2020/4
1,Equitas Selection FIC FIA,2020-04-30,"3,018.52",4.85,"14,640.05",0.00,0,"14,640.05",0,"14,640.05",2020/4
2,Polo Norte I FIC FIM,2020-04-30,"3,946.89",8.10,"31,975.51",296.32,0,"31,679.19",0,"31,975.51",2020/4
3,Mauá Macro FIC FIM,2020-04-30,37.17,775.21,"28,816.89",116.98,0,"28,699.91",0,"28,816.89",2020/4
4,Bahia AM Maraú FIC de FIM,2020-04-30,"33,810.43",2.39,"80,837.41",0.00,0,"80,837.41",0,"80,837.41",2020/4


,Papel,Qtd Disponivel,Qtd Projetada,Qtd Dia,Qtde Total,Ult Cotacao,Financeiro,period
0,RNGO11,49,0,0,49.00,73.62,"3,607.38",2020/4
1,OULG11,50,0,0,50.00,95.50,"4,775.00",2020/4
2,CNES11,11,0,0,11.00,70.00,770.00,2020/4
3,IBFF11,50,0,0,50.00,76.49,"3,824.50",2020/4
4,BCFF11,8,0,0,8.00,85.13,681.04,2020/4


In [6]:
df_abr[~df_abr['Unnamed: 2'].isnull()].head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
16,nan,NaN,Ações,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,38128,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
20,nan,NaN,Papel,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
24,nan,NaN,MGLU3,nan,nan,nan,nan,nan,nan,nan,800,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,NaN,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,NaN,800,nan,NaN,nan,NaN,nan,NaN,nan,47.66,NaN,nan,nan,nan,nan,NaN,"38,128.00",NaN
27,nan,NaN,Opções,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,0,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
29,nan,NaN,Termos,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,0,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
40,nan,NaN,Aluguel (BTC),nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,0,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
42,nan,NaN,Mercadorias e futuros,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
44,nan,NaN,Ouro Disponível,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,0,nan,NaN,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
46,nan,NaN,Proventos de Ação,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,NaN,nan,NaN,nan,nan,

In [7]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
0,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
1,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,Histórico de Posição Consolidada,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
2,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,Mês: Maio/2019,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
3,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,Data de referência: 31/05/2019,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
4,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
5,nan,Gabriel Machado Lopes Goncalves,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,CPF/CNPJ,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,980.742.371-68,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
6,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
7,nan,"Rua Barata Ribeiro, 160 - Ap 908\nRio De Janei...",NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN,nan,Conta XP,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,200091,nan,nan,NaN,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,nan,nan,NaN,nan,nan
8,nan,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,NaN,nan,NaT,NaN,nan,nan,nan,nan,NaN,nan,